In [27]:
import torch
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CyclicLR, OneCycleLR, StepLR
import numpy as np
import math

In [35]:
fake_model = torch.nn.Linear(2, 1)
fake_optimizer = torch.optim.SGD(fake_model.parameters(), 8)
levels_of_k = []

num_retrain=50
step_size = math.floor(num_retrain/5)
high = 2 + 6
low = -6 + 6

print(step_size)

#weight_noise_schedule = [ "cawr", "cyclic", "onecycle", "step" ]

scheduler = CosineAnnealingWarmRestarts(fake_optimizer, 
                T_0 = 10,# Number of iterations for the first restart
                T_mult = 1, # A factor increases TiTi​ after a restart
                eta_min = low) # Minimum learning rate

scheduler = CyclicLR(fake_optimizer, 
                base_lr = 0, # Initial learning rate which is the lower boundary in the cycle for each parameter group
                max_lr = 8, # Upper learning rate boundaries in the cycle for each parameter group
                step_size_up = 5, # Number of training iterations in the increasing half of a cycle
                mode = "triangular")

scheduler = OneCycleLR(fake_optimizer, 
                max_lr = 8, # Upper learning rate boundaries in the cycle for each parameter group
                steps_per_epoch = 1, # The number of steps per epoch to train for.
                epochs = num_retrain, # The number of epochs to train for.
                anneal_strategy = 'cos') # Specifies the annealing strategy

scheduler = StepLR(fake_optimizer, 
                step_size = step_size, # Period of learning rate decay
                gamma = 0.5) # Multiplicative factor of learning rate decay

if args.scheduler == "base":
    scheduler = StepLR(fake_optimizer, 
                step_size = 5, # Period of learning rate decay
                gamma = 0.5) # Mu

for i in range(num_retrain):
    fake_optimizer.step()
    levels_of_k.append( round (fake_optimizer.param_groups[0]["lr"],6 ) )
    scheduler.step()

# if args.scheduler == "base":
#     levels_of_k = []
#     levels_of_k = [args.rank_weight_k for i in range(num_retrain)]
    
print(levels_of_k)
new_ks = [10 ** x for x in list(levels_of_k - np.full_like(levels_of_k, 6))]
print(new_ks)
# print(args.scheduler)
# print(args.rank_weight_k)

10
[8.0, 7.804226, 7.236068, 6.351141, 5.236068, 4.0, 2.763932, 1.648859, 0.763932, 0.195774, 8.0, 7.804226, 7.236068, 6.351141, 5.236068, 4.0, 2.763932, 1.648859, 0.763932, 0.195774, 8.0, 7.804226, 7.236068, 6.351141, 5.236068, 4.0, 2.763932, 1.648859, 0.763932, 0.195774, 8.0, 7.804226, 7.236068, 6.351141, 5.236068, 4.0, 2.763932, 1.648859, 0.763932, 0.195774, 8.0, 7.804226, 7.236068, 6.351141, 5.236068, 4.0, 2.763932, 1.648859, 0.763932, 0.195774]
[100.0, 63.712698549236954, 17.221381990198296, 2.244610550854646, 0.17221381990198295, 0.01, 0.0005806734909946019, 4.4551158312039714e-05, 5.806734909946025e-06, 1.5695458248832506e-06, 100.0, 63.712698549236954, 17.221381990198296, 2.244610550854646, 0.17221381990198295, 0.01, 0.0005806734909946019, 4.4551158312039714e-05, 5.806734909946025e-06, 1.5695458248832506e-06, 100.0, 63.712698549236954, 17.221381990198296, 2.244610550854646, 0.17221381990198295, 0.01, 0.0005806734909946019, 4.4551158312039714e-05, 5.806734909946025e-06, 1.569545

In [22]:
print("One Cycle")
print([4.0, 5.20346, 8.753494, 14.472089, 22.07249, 31.173581, 41.318995, 52.0, 62.681005, 72.826419, 81.92751, 89.527911, 95.246506, 98.79654, 100.0, 99.798716, 99.196483, 98.19815, 96.811756, 95.048463, 92.922468, 90.450888, 87.653623, 84.553194, 81.174565, 77.544939, 73.693538, 69.651373, 65.450988, 61.126202, 56.711836, 52.243433, 47.756967, 43.288564, 38.874198, 34.549412, 30.349027, 26.306862, 22.455461, 18.825835, 15.447206, 12.346777, 9.549512, 7.077932, 4.951937, 3.188644, 1.80225, 0.803917, 0.201684, 0.0004])
print([2.0892961308540407e-06, 2.6078163268324754e-06, 5.015104651644385e-06, 1.4380268665020904e-05, 5.83175136919719e-05, 0.0003118070999548103, 0.002020784490414368, 0.01445439770745928, 0.10339034868708946, 0.6700604736572178, 3.582622095121406, 14.528908878706261, 41.66007044676503, 80.11669032656748, 100.0, 96.36004629992512, 86.24203584554118, 71.7549721145981, 55.58287414055304, 40.167703258534225, 27.151698345337284, 17.22150095170151, 10.287029849193924, 5.811068568683737, 3.1186382414187817, 1.5980670609402206, 0.7861094191207535, 0.3733447081967071, 0.1722181818639431, 0.07764187235984618, 0.03443079098915216, 0.015117326745280409, 0.006615413573367626, 0.002904591084627416, 0.0012880597222109478, 0.0005807015697260693, 0.00026786871850094596, 0.00012721812779653173, 6.258020766970958e-05, 3.206764004095084e-05, 1.7209807002228142e-05, 9.721695184115545e-06, 5.807122667428158e-06, 3.6832822489328936e-06, 2.489745751706961e-06, 1.7992478814764361e-06, 1.3937343377967192e-06, 1.1596127985994424e-06, 1.0378509806075006e-06, 1.0000736854376144e-06])

One Cycle
[4.0, 5.20346, 8.753494, 14.472089, 22.07249, 31.173581, 41.318995, 52.0, 62.681005, 72.826419, 81.92751, 89.527911, 95.246506, 98.79654, 100.0, 99.798716, 99.196483, 98.19815, 96.811756, 95.048463, 92.922468, 90.450888, 87.653623, 84.553194, 81.174565, 77.544939, 73.693538, 69.651373, 65.450988, 61.126202, 56.711836, 52.243433, 47.756967, 43.288564, 38.874198, 34.549412, 30.349027, 26.306862, 22.455461, 18.825835, 15.447206, 12.346777, 9.549512, 7.077932, 4.951937, 3.188644, 1.80225, 0.803917, 0.201684, 0.0004]
[2.0892961308540407e-06, 2.6078163268324754e-06, 5.015104651644385e-06, 1.4380268665020904e-05, 5.83175136919719e-05, 0.0003118070999548103, 0.002020784490414368, 0.01445439770745928, 0.10339034868708946, 0.6700604736572178, 3.582622095121406, 14.528908878706261, 41.66007044676503, 80.11669032656748, 100.0, 96.36004629992512, 86.24203584554118, 71.7549721145981, 55.58287414055304, 40.167703258534225, 27.151698345337284, 17.22150095170151, 10.287029849193924, 5.8110685

In [21]:
print("Cylic")
print([1e-06, 20.000001, 40.000001, 60.0, 80.0, 100.0, 80.0, 60.0, 40.000001, 20.000001, 1e-06, 20.000001, 40.000001, 60.0, 80.0, 100.0, 80.0, 60.0, 40.000001, 20.000001, 1e-06, 20.000001, 40.000001, 60.0, 80.0, 100.0, 80.0, 60.0, 40.000001, 20.000001, 1e-06, 20.000001, 40.000001, 60.0, 80.0, 100.0, 80.0, 60.0, 40.000001, 20.000001, 1e-06, 20.000001, 40.000001, 60.0, 80.0, 100.0, 80.0, 60.0, 40.000001, 20.000001])
print([1e-06, 3.9810717055349695e-05, 0.001584893192461114, 0.0630957344480193, 2.5118864315095824, 100.0, 2.5118864315095824, 0.0630957344480193, 0.001584893192461114, 3.9810717055349695e-05, 1e-06, 3.9810717055349695e-05, 0.001584893192461114, 0.0630957344480193, 2.5118864315095824, 100.0, 2.5118864315095824, 0.0630957344480193, 0.001584893192461114, 3.9810717055349695e-05, 1e-06, 3.9810717055349695e-05, 0.001584893192461114, 0.0630957344480193, 2.5118864315095824, 100.0, 2.5118864315095824, 0.0630957344480193, 0.001584893192461114, 3.9810717055349695e-05, 1e-06, 3.9810717055349695e-05, 0.001584893192461114, 0.0630957344480193, 2.5118864315095824, 100.0, 2.5118864315095824, 0.0630957344480193, 0.001584893192461114, 3.9810717055349695e-05, 1e-06, 3.9810717055349695e-05, 0.001584893192461114, 0.0630957344480193, 2.5118864315095824, 100.0, 2.5118864315095824, 0.0630957344480193, 0.001584893192461114, 3.9810717055349695e-05])

Cylic
[1e-06, 20.000001, 40.000001, 60.0, 80.0, 100.0, 80.0, 60.0, 40.000001, 20.000001, 1e-06, 20.000001, 40.000001, 60.0, 80.0, 100.0, 80.0, 60.0, 40.000001, 20.000001, 1e-06, 20.000001, 40.000001, 60.0, 80.0, 100.0, 80.0, 60.0, 40.000001, 20.000001, 1e-06, 20.000001, 40.000001, 60.0, 80.0, 100.0, 80.0, 60.0, 40.000001, 20.000001, 1e-06, 20.000001, 40.000001, 60.0, 80.0, 100.0, 80.0, 60.0, 40.000001, 20.000001]
[1e-06, 3.9810717055349695e-05, 0.001584893192461114, 0.0630957344480193, 2.5118864315095824, 100.0, 2.5118864315095824, 0.0630957344480193, 0.001584893192461114, 3.9810717055349695e-05, 1e-06, 3.9810717055349695e-05, 0.001584893192461114, 0.0630957344480193, 2.5118864315095824, 100.0, 2.5118864315095824, 0.0630957344480193, 0.001584893192461114, 3.9810717055349695e-05, 1e-06, 3.9810717055349695e-05, 0.001584893192461114, 0.0630957344480193, 2.5118864315095824, 100.0, 2.5118864315095824, 0.0630957344480193, 0.001584893192461114, 3.9810717055349695e-05, 1e-06, 3.981071705534969

In [34]:
print("Step")
print([100.0, 100.0, 100.0, 100.0, 100.0, 50.0, 50.0, 50.0, 50.0, 50.0, 25.0, 25.0, 25.0, 25.0, 25.0, 12.5, 12.5, 12.5, 12.5, 12.5, 6.25, 6.25, 6.25, 6.25, 6.25, 3.125, 3.125, 3.125, 3.125, 3.125, 1.5625, 1.5625, 1.5625, 1.5625, 1.5625, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.390625, 0.390625, 0.390625, 0.390625, 0.390625, 0.195312, 0.195312, 0.195312, 0.195312, 0.195312])
print([100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06])

Step
[100.0, 100.0, 100.0, 100.0, 100.0, 50.0, 50.0, 50.0, 50.0, 50.0, 25.0, 25.0, 25.0, 25.0, 25.0, 12.5, 12.5, 12.5, 12.5, 12.5, 6.25, 6.25, 6.25, 6.25, 6.25, 3.125, 3.125, 3.125, 3.125, 3.125, 1.5625, 1.5625, 1.5625, 1.5625, 1.5625, 0.78125, 0.78125, 0.78125, 0.78125, 0.78125, 0.390625, 0.390625, 0.390625, 0.390625, 0.390625, 0.195312, 0.195312, 0.195312, 0.195312, 0.195312]
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06, 3.162277660168379e-06]


In [38]:
print("Cosine Annealing")
print([100.0, 97.552826, 90.45085, 79.389263, 65.45085, 50.0, 34.549151, 20.610738, 9.549151, 2.447175, 100.0, 97.552826, 90.45085, 79.389263, 65.45085, 50.0, 34.549151, 20.610738, 9.549151, 2.447175, 100.0, 97.552826, 90.45085, 79.389263, 65.45085, 50.0, 34.549151, 20.610738, 9.549151, 2.447175, 100.0, 97.552826, 90.45085, 79.389263, 65.45085, 50.0, 34.549151, 20.610738, 9.549151, 2.447175, 100.0, 97.552826, 90.45085, 79.389263, 65.45085, 50.0, 34.549151, 20.610738, 9.549151, 2.447175])
print([100.0, 63.712698549236954, 17.221381990198296, 2.244610550854646, 0.17221381990198295, 0.01, 0.0005806734909946019, 4.4551158312039714e-05, 5.806734909946025e-06, 1.5695458248832506e-06, 100.0, 63.712698549236954, 17.221381990198296, 2.244610550854646, 0.17221381990198295, 0.01, 0.0005806734909946019, 4.4551158312039714e-05, 5.806734909946025e-06, 1.5695458248832506e-06, 100.0, 63.712698549236954, 17.221381990198296, 2.244610550854646, 0.17221381990198295, 0.01, 0.0005806734909946019, 4.4551158312039714e-05, 5.806734909946025e-06, 1.5695458248832506e-06, 100.0, 63.712698549236954, 17.221381990198296, 2.244610550854646, 0.17221381990198295, 0.01, 0.0005806734909946019, 4.4551158312039714e-05, 5.806734909946025e-06, 1.5695458248832506e-06, 100.0, 63.712698549236954, 17.221381990198296, 2.244610550854646, 0.17221381990198295, 0.01, 0.0005806734909946019, 4.4551158312039714e-05, 5.806734909946025e-06, 1.5695458248832506e-06])

Cosine Annealing
[100.0, 97.552826, 90.45085, 79.389263, 65.45085, 50.0, 34.549151, 20.610738, 9.549151, 2.447175, 100.0, 97.552826, 90.45085, 79.389263, 65.45085, 50.0, 34.549151, 20.610738, 9.549151, 2.447175, 100.0, 97.552826, 90.45085, 79.389263, 65.45085, 50.0, 34.549151, 20.610738, 9.549151, 2.447175, 100.0, 97.552826, 90.45085, 79.389263, 65.45085, 50.0, 34.549151, 20.610738, 9.549151, 2.447175, 100.0, 97.552826, 90.45085, 79.389263, 65.45085, 50.0, 34.549151, 20.610738, 9.549151, 2.447175]
[100.0, 63.712698549236954, 17.221381990198296, 2.244610550854646, 0.17221381990198295, 0.01, 0.0005806734909946019, 4.4551158312039714e-05, 5.806734909946025e-06, 1.5695458248832506e-06, 100.0, 63.712698549236954, 17.221381990198296, 2.244610550854646, 0.17221381990198295, 0.01, 0.0005806734909946019, 4.4551158312039714e-05, 5.806734909946025e-06, 1.5695458248832506e-06, 100.0, 63.712698549236954, 17.221381990198296, 2.244610550854646, 0.17221381990198295, 0.01, 0.0005806734909946019, 4.4551